In [1]:
import sklearn.metrics
import numpy as np
import pandas as pd

In [2]:
import transparentai.models.classification as classification
import transparentai.models.regression as regression

In [13]:
import transparentai.models.evaluation.classification as clf_eval

dir(clf_eval)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'accuracy',
 'average_precision',
 'balanced_accuracy',
 'brier_score',
 'confusion_matrix',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'false_negatives',
 'false_positives',
 'jaccard',
 'log_loss',
 'matthews_corrcoef',
 'np',
 'precision',
 'recall',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'sklearn',
 'true_negatives',
 'true_positives']

In [10]:
y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0, 1, 0, 1, 0, 1, 0, 1]
metrics = ['accuracy','f1',lambda y_true, y_pred:1]

print(classification.compute_metrics(y_true, y_pred, metrics))


y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0.2, 0.49, 0, 1, 0.3, 0.8, 0, 1]

print(classification.compute_metrics(y_true, y_pred, metrics))

y_true = [0, 0, 0, 1, 2, 1, 1, 1, 2]
y_pred = [0, 1, 0, 1, 1, 1, 0, 1, 2]

metrics = ['accuracy','f1_macro',lambda y_true, y_pred:1]
print(classification.compute_metrics(y_true, y_pred, metrics))
    

y_true = [3, 5, 2.5, 7, 9, 5.5]
y_pred = [2.5, 5, 4, 8, 9, 4.5]
metrics = ['RMSE','MAE','mean_squared_log_error',lambda y_true, y_pred:1]

print(regression.compute_metrics(y_true, y_pred, metrics))

{'accuracy': 0.625, 'f1': 0.6666666666666665, 'custom_1': 1}
{'accuracy': 0.75, 'f1': 0.7499999999999999, 'custom_1': 1}
{'accuracy': 0.6666666666666666, 'f1_macro': 0.6666666666666665, 'custom_1': 1}
{'RMSE': 0.8660254037844386, 'MAE': 0.6666666666666666, 'mean_squared_log_error': 0.031137926523503903, 'custom_1': 1}
